# Applying the perceptron theory

Now, we are diving into the heart of it! Programming our first neural network using `pytorch`.

Note, run all the cells every time to prevent unwanted errors.


In [1]:
import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# Load dataset
filename = "./data/iris.csv"
df = pd.read_csv(filename)
df = df.sample(frac=1).reset_index(drop=True) # Shuffle dataframe
df.head()
print(df.shape)
print(df["species"].value_counts())

(150, 5)
Iris-versicolor    50
Iris-virginica     50
Iris-setosa        50
Name: species, dtype: int64


In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,6.7,2.5,5.8,1.8,Iris-virginica
1,4.9,3.1,1.5,0.1,Iris-setosa
2,5.7,3.8,1.7,0.3,Iris-setosa
3,5.0,3.0,1.6,0.2,Iris-setosa
4,6.2,2.2,4.5,1.5,Iris-versicolor


In [4]:
# Creating a test/train split
train_test_split_fraction = 0.80
split_index = int(df.shape[0] * train_test_split_fraction)
df_train = df[:split_index]
df_test = df[split_index:]

target = pd.get_dummies(df['species']).values # One hot encode
target[:5]

array([[0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [5]:
# Selecting the features and the target
X_train = df_train.drop('species', axis = 1).values
X_test = df_test.drop('species', axis = 1).values

y_train = target[:split_index]
y_test = target[split_index:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(120, 4)
(30, 4)
(120, 3)
(30, 3)


Transform `X_train`, `y_train` and their test counterparts into pytorch tensors using `torch.tensor()`. Make sure to convert them to float32 with numpy.

In [7]:
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train) 
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [8]:
X_train[:5]

tensor([[6.7000, 2.5000, 5.8000, 1.8000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [5.7000, 3.8000, 1.7000, 0.3000],
        [5.0000, 3.0000, 1.6000, 0.2000],
        [6.2000, 2.2000, 4.5000, 1.5000]], dtype=torch.float64)

In [9]:
y_train[:5]

tensor([[0, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [0, 1, 0]], dtype=torch.uint8)

Now that we have selected our train and test sets, we have to create the neural network architecture.

In [10]:
nb_hidden_neurons = 10
nb_classes = len(pd.unique(df['species']))
nb_classes

3

The `nn.Linear()` method defines a layer of neurons in the neural network.
We want the network to have 10 neurons in the middle layers and 3 in the last layer (one for every class).

Notice how we define the layers in `__init__()`, and we connect them in `forward()` .

In the code below in the `__init__()` function,  complete the code for `self.layer_1`, `self.layer_2` , and `self.layer_3` using the `nn.Linear()` class to add the desired layers. *Hint: Make use of `nb_features` and `nb_hidden_neurons`. You can also refer to the architecture of the network shown in the image a few cells below*.

In the `forward()` function, which is going to handle the forward pass of the neural network learning process, we will use the ReLU activation function for the first layers and for the last layer, we are going to use the Softmax activation function.
We want you to assign the ReLU activation function to the variable `activation_function`. More information [here](https://pytorch.org/docs/stable/nn.html).
Lastly, we want you to write the code to combine the layers to the input `x` and the output. *Hint: You can refer yourself to how the last layer code is written*.


In [16]:
import torch.nn as nn

class Network(nn.Module):

    def __init__(self, nb_features):
        """Here we define the layers
        """

        super().__init__()
        
        self.layer_1 =  nn.Linear(nb_features,nb_hidden_neurons)
        self.layer_2 =  nn.Linear(nb_hidden_neurons, nb_hidden_neurons)
        self.layer_3 =  nn.Linear(nb_hidden_neurons, nb_classes)
        
        

    def forward(self,x):
        """Here we combine the layers
        """
        
        activation_function = self.sigmoid()
        last_layer_activation = nn.Softmax(dim=1)
        
        output_first_layer = nb_hidden_neurons 
        output_second_layer = nb_hidden_neurons
        prediction = last_layer_activation(self.layer_3(output_second_layer))
        return prediction

In [13]:
import numpy as np
import torch as tr
import torch.nn as nn
class Module1(nn.Module):
    def __init__(self, dIn, dOut, numLayers):
        super(Module1, self).__init__()
        self.layers = []
        for i in range(numLayers - 1):
            self.layers.append(nn.Conv2d(in_channels=dIn, out_channels=dIn, kernel_size=1))
        self.layers.append(nn.Conv2d(in_channels=dIn, out_channels=dOut, kernel_size=1))

    def forward(self, x):
        y = x
        for i in range(len(self.layers)):
            y = self.layers[i](y)
        return y

In [18]:
X_train.shape[1]


4

In [20]:
my_nn = Network(nb_features=X_train.shape[1])
my_nn

Network(
  (layer_1): Linear(in_features=4, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=3, bias=True)
)

If you did everything right, this should be the output of the previous call

```
Network(
  (layer_1): Linear(in_features=4, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=10, bias=True)
  (layer_3): Linear(in_features=10, out_features=3, bias=True)
)
```

Now, we have done the architecture of the model. Right now, it does nothing. We have to make it learn.

First, we'll need to define the `criterion` by which the cost if calculated.
There are many possible losses to choose from (see #Additional reading material), but here we're going to choose the `MSELoss`.
The piece of code that is going to compute the gradient and change the weights is called an `optimizer`.
Again, there are many to choose from (see #Additional reading material), but here we are going to use the `Adam` optimizer with a `learning_rate` of `0.001`.

In [29]:
# Select your criterion, your learning rate and your optimizer.
import torch.optim as optim
criterion = torch.nn.MSELoss(size_average=None, reduce=None, reduction='mean')
learning_rate = 0.001
optimizer = optim.Adam([X_train, y_train], lr=0.0001)

Now, we can start training the neural network. Below, the `training()` function handles all the necessary stuff for an efficient training process.
The loops, the input and output getting, and the loss plotting are already taken care of.
You just have to do the most important part of the training process, which is listed as a comment.

You can run the function with the next cell, which will also plot a graph of the loss over the whole training.

In [41]:
def training(batch_size : int, nb_steps_loss_sum : int):
    """ Train the neural network, feeding it `batch_size` at a time
    and saving statistics every `nb_steps_loss_sum` steps.
    
    Arguments:
    
    - batch_size [int] : the number of input samples at each training step (called a batch)
    - nb_steps_loss_sum [int] : the number of batches before saving the loss for plotting
    
    Returns:
    - loss_list : [List[double]] : value of the loss every `nb_steps_loss_sum` steps
    """

    loss_list = []
    running_loss = 0
    batch_nb = 0

    for epoch in range(0,10): # Number of times to iterate through the complete dataset
        for idx in range(0, X_train.shape[0], batch_size):
            
            # Get input and output
            input_batch = X_train[idx:idx + batch_size]
            target = y_train[idx:idx + batch_size]
            
            # TO COMPLETE:
            # - zero gradient buffers
            optimizer.zero_grad()
            # - compute the forward pass
            outputs = Network(input_batch)
            # - compute the loss
            loss = criterion(target, labels)
            # - backpropagate
            loss.backward()
            # - do a step
            optimizer.step()
          
            
            # Save the loss every `running_loss_steps` batches
            running_loss += loss.item()
            save_loss_condition = batch_nb % nb_steps_loss_sum == (nb_steps_loss_sum - 1)
            if save_loss_condition:    
                loss_list.append(running_loss)
                running_loss = 0.0


            batch_nb+= 1
        
    return loss_list


In [42]:
nb_steps_loss_sum = 10
loss = training(batch_size=1, nb_steps_loss_sum=nb_steps_loss_sum)

# Plotting the loss over training
plt.figure()
plt.plot(range(0, len(loss)), loss)
plt.xlabel(f"Batches/{nb_steps_loss_sum}")
plt.ylabel("Loss")
plt.title("Training loss")
plt.show()
plt.close()

tensor([[0, 0, 1]], dtype=torch.uint8)


NameError: name 'labels' is not defined

If everything went to plan, you should have a decreasing loss over time. Something similar to this:
    
![Training Loss](./assets/loss.png)

The function below will compute the accuracy of the neural network. You have to complete the steps outlined.

In [ ]:
def computeScore(X, y):
    correct = 0
    total = 0
    batch_size = 1
    my_nn.eval()
    with torch.no_grad():
        for idx in range(0, X.shape[0], batch_size):
            # TO COMPLETE:
            # - get the `batch_size` number of input samples
            # - compute the prediction of the neural network
            # - get the max of the prediction (e.g. get the most likely class)
            # This can be done using `torch.max`.
            # - get the max of the target (e.g. correct class)
            # - check if the prediction is correct and count it
            # - count every sample
            
    accuracy = correct/total * 100
    print(f"Accuracy of the network on the {total} samples: {accuracy:.2f}%")

In [ ]:
computeScore(X_train, y_train)
computeScore(X_test, y_test)

If you compute the score, and have done everything correctly (and you are lucky), you should have an accuracy of 90% and above.
```
Accuracy of the network on the 120 samples: 96.67%
Accuracy of the network on the 30 samples: 96.67%
```

However, you might notice, if you repeat the runs, you might get totally different scores, even if all else stays the same.

**Why does that happen ? Can you explain ?** *Hint: It might have something to do with degrees of freedom*

> Explain here

## Additional reading material
[How to build a simple neural network in pytorch](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html)

[Introduction to optimizers](https://algorithmia.com/blog/introduction-to-optimizers)

[What loss function to choose](https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/)

[Visualizing optimizers (Interactive)](https://emiliendupont.github.io/2018/01/24/optimization-visualization/)

[What are degrees of freedom?](https://machinelearningmastery.com/degrees-of-freedom-in-machine-learning/)